<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install sodapy

In [0]:
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
%matplotlib inline


In [73]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=200000, select='trip_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tip, additional_charges, trip_total' )

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,ecc1cd019a7245f5d1ee0b6a6de54a7a747891c3,2019-03-11T08:45:00.000,2019-03-11T08:45:00.000,735,2.85320448622395,8,24,7.5,0,2.55,10.05
1,ecc1cd815dbee6b5f9cf91e5d8209500e8071853,2019-03-27T19:30:00.000,2019-03-27T19:45:00.000,1015,6.76922566128,28,21,10,0,0,10
2,ecc1cde0a29114f08b85de6ebd7a5f2f72a52ea1,2018-11-18T15:00:00.000,2018-11-18T15:15:00.000,753,3.4464622883569,28,32,7.5,0,2.5,10
3,ecc1cf1cbb2db0c04cf39479d018b2688baf7268,2019-01-23T18:45:00.000,2019-01-23T19:00:00.000,892,2.95365184688,69,40,7.5,0,0,7.5
4,ecc1cf5253f0aefc61e1bbb83109fe5a185316f7,2018-12-14T18:45:00.000,2018-12-14T19:30:00.000,2512,10.67242095904,NaN,24,15,0,0,15
...,...,...,...,...,...,...,...,...,...,...,...
199995,ee4cd31f57f2c38057c88617afcb9f1f4e471386,2019-02-03T18:30:00.000,2019-02-03T19:30:00.000,4035,49.46450235808,NaN,8,57.5,5,2.62,65.12
199996,ee4cd33d974a321a4dff1ed61a88b66bab0f372f,2018-12-26T12:30:00.000,2018-12-26T12:45:00.000,819,2.84987321056,25,25,7.5,0,2.5,10
199997,ee4cd36d029e40d566f5b3ca5397f41be6b4666f,2019-01-20T10:00:00.000,2019-01-20T10:00:00.000,199,0.75078237525797,3,6,2.5,0,2.55,5.05
199998,ee4cd3fe3b58a4513e6d229c9310636a8fb3e03d,2019-01-01T01:45:00.000,2019-01-01T02:00:00.000,1288,5.73521508958714,8,5,5,0,2.5,7.5


In [75]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].fillna('0')
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df['start_date_plus_hour'] = df['trip_start_timestamp'].apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

weather_df = pd.read_csv('https://github.com/muoyo/chicago-ridesharing/raw/master/data/chicago_weather.csv')
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))
weather_df['start_date_plus_hour'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df = weather_df.rename(columns={'icon': 'precipitation'})

# def set_precip(precip):
#     if precip not in ['rain', 'snow']: 
#         precip = 'clear'

#     return precip

# weather_df['precipitation'] = weather_df['precipitation'].apply(set_precip)
precip_df = weather_df[['start_date_plus_hour', 'precipitation']]

df = df.merge(precip_df, how='left', on='start_date_plus_hour')
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,start_weekday,start_hour,start_time_block,start_date_plus_hour,precipitation
0,ecc1cd019a7245f5d1ee0b6a6de54a7a747891c3,2019-03-11 08:45:00,2019-03-11 08:45:00,735,2.853204,8,7.5,0.0,2.55,10.05,0,8,2,2019-03-11 08:00:00,partly-cloudy-day
1,ecc1cd815dbee6b5f9cf91e5d8209500e8071853,2019-03-27 19:30:00,2019-03-27 19:45:00,1015,6.769226,28,10.0,0.0,0.00,10.00,2,19,6,2019-03-27 19:00:00,partly-cloudy-day
2,ecc1cde0a29114f08b85de6ebd7a5f2f72a52ea1,2018-11-18 15:00:00,2018-11-18 15:15:00,753,3.446462,28,7.5,0.0,2.50,10.00,6,15,5,2018-11-18 15:00:00,partly-cloudy-day
3,ecc1cf1cbb2db0c04cf39479d018b2688baf7268,2019-01-23 18:45:00,2019-01-23 19:00:00,892,2.953652,69,7.5,0.0,0.00,7.50,2,18,6,2019-01-23 18:00:00,partly-cloudy-night
4,ecc1cf5253f0aefc61e1bbb83109fe5a185316f7,2018-12-14 18:45:00,2018-12-14 19:30:00,2512,10.672421,NaN,15.0,0.0,0.00,15.00,4,18,6,2018-12-14 18:00:00,cloudy


In [25]:
columns_to_use = ['start_weekday', 'start_hour', 'pickup_community_area']

columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
X_cat = df.drop(columns=columns_to_drop)
X_cat = X_cat.fillna('0')
X_cat

,pickup_community_area,start_weekday,start_hour
0,0,0,23
1,15,0,23
2,7,0,23
3,2,0,23
4,8,0,23
...,...,...,...
199995,53,0,6
199996,8,0,6
199997,32,0,6
199998,19,0,6


In [26]:
enc = OneHotEncoder()
# X_cat_ohe = enc.fit_transform(X_cat)
# columns = enc.get_feature_names(input_features=X_cat.columns)
# X_cat_df = pd.DataFrame(X_cat_ohe.todense(), columns=columns)
y = df['trip_total']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.25, random_state=12)

# Transform training and test sets
X_train_enc = enc.fit_transform(X_train, y_train)
X_test_enc = enc.transform(X_test)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train.columns)
X_train_df = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
X_test_df = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

X_train_df = sm.add_constant(X_train_df)
model = sm.OLS(y_train,X_train_df).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     720.5
Date:                Wed, 22 Jan 2020   Prob (F-statistic):               0.00
Time:                        13:16:07   Log-Likelihood:            -5.6405e+05
No. Observations:              150000   AIC:                         1.128e+06
Df Residuals:                  149905   BIC:                         1.129e+06
Df Model:                          94                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
pickup_community_area_0     14.5001      0.116    125.103      0.000      14.273      14.727
pickup_community_area_1      0.1368      0.265      0.517      0.605      -0.382       0.655
pickup_community_area_10     0.7290      0.494      1.475      0.140      -0.240       1.697
pickup_community_area_11     0.8342      0.488      1.710      0.087      -0.122       1.790
pickup_community_area_12     1.5581      0.692      2.253      0.024       0.203       2.914
pickup_community_area_13     0.2849      0.508      0.561      0.575      -0.711       1.281
pickup_community_area_14    -0.0236      0.338     -0.070      0.944      -0.687       0.640
pickup_community_area_15    -0.1720      0.331     -0.519      0.604      -0.821       0.477
pickup_community_area_16     0.2527      0.286      0.884      0.376      -0.307       0.813
pickup_community_area_17    -0.3872      0.483     -0.801      0.423      -1.335       0.560
pickup_community_area_18    -0.0082      0.832     -0.010      0.992      -1.639       1.623
pickup_community_area_19    -0.5180      0.303     -1.708      0.088      -1.112       0.077
pickup_community_area_2     -0.8937      0.299     -2.992      0.003      -1.479      -0.308
pickup_community_area_20    -1.3781      0.492     -2.801      0.005      -2.342      -0.414
pickup_community_area_21    -0.4928      0.278     -1.771      0.077      -1.038       0.053
pickup_community_area_22    -0.8150      0.169     -4.832      0.000      -1.146      -0.484
pickup_community_area_23    -1.8133      0.302     -5.995      0.000      -2.406      -1.220
pickup_community_area_24    -1.4303      0.130    -11.024      0.000      -1.685      -1.176
pickup_community_area_25    -1.1295      0.269     -4.198      0.000      -1.657      -0.602
pickup_community_area_26    -2.1456      0.549     -3.909      0.000      -3.221      -1.070
pickup_community_area_27    -1.4906      0.423     -3.521      0.000      -2.320      -0.661
pickup_community_area_28    -1.4263      0.107    -13.338      0.000      -1.636      -1.217
pickup_community_area_29    -1.5712      0.341     -4.611      0.000      -2.239      -0.903
pickup_community_area_3      0.4142      0.200      2.070      0.038       0.022       0.806
pickup_community_area_30    -0.9581      0.327     -2.933      0.003      -1.598      -0.318
pickup_community_area_31    -1.7125      0.255     -6.718      0.000      -2.212      -1.213
pickup_community_area_32     0.4999      0.102      4.890      0.000       0.300       0.700
pickup_community_area_33     0.1673      0.214      0.781      0.435      -0.252       0.587
pickup_community_area_34    -1.6980      0.414     -4.104      0.000      -2.509      -0.887
pickup_community_area_35    -1.6914      0.314     -5.381      0.000      -2.307      -1.075
pickup_communi

In [27]:

# Fit the model
linreg = LinearRegression()
linreg.fit(X_train_df, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train_df, y_train))
print('Test r^2:', linreg.score(X_test_df, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_df)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_df)))


Training r^2: 0.31120337885173954
Test r^2: 0.3158165059278203
Training MSE: 108.06877067877224
Test MSE: 104.45088889944266


In [71]:
precip_df.loc[precip_df['precipitation'] == 'rain']
precip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Data columns (total 2 columns):
start_date_plus_hour    10223 non-null datetime64[ns]
precipitation           10223 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 159.9+ KB
